<a href="https://colab.research.google.com/github/strangelycutlemon/DS-Unit-2-Regression-Classification/blob/master/module4/assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4

## Assignment

- [X] Watch Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [x] Do train/validate/test split with the Tanzania Waterpumps data.
- [X] Do one-hot encoding. For example, in addition to `quantity`, you could try `basin`, `extraction_type_class`, and more. (But remember it may not work with high cardinality categoricals.)
- [X] Use scikit-learn for logistic regression.
- [X] Get your validation accuracy score.
- [X] Get and plot your coefficients.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

> [Do Not Copy-Paste.](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) You must type each of these exercises in, manually. If you copy and paste, you might as well not even do them. The point of these exercises is to train your hands, your brain, and your mind in how to read, write, and see code. If you copy-paste, you are cheating yourself out of the effectiveness of the lessons.


## Stretch Goals

### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

To visualize this dataset, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from the previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```

#### Pipelines

[Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/compose.html) explains why pipelines are useful, and demonstrates how to use them:

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:
> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

### Reading
- [ ] [Why is logistic regression considered a linear model?](https://www.quora.com/Why-is-logistic-regression-considered-a-linear-model)
- [ ] [Training, Validation, and Testing Data Sets](https://end-to-end-machine-learning.teachable.com/blog/146320/training-validation-testing-data-sets)
- [ ] [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)
- [ ] [Always start with a stupid model, no exceptions](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)
- [ ] [Statistical Modeling: The Two Cultures](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).



In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module4')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [5]:
# make a validation set from train set

from sklearn.model_selection import train_test_split

X_train = train_features
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.8, test_size=0.2,
    random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [6]:
# Make numeric subsets
X_train_numeric = X_train.select_dtypes('number')
X_val_numeric = X_val.select_dtypes('number')

X_train_numeric.isnull().sum()

id                   0
amount_tsh           0
gps_height           0
longitude            0
latitude             0
num_private          0
region_code          0
district_code        0
population           0
construction_year    0
dtype: int64

In [7]:
# Check cardinality of categorical features

X_train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,47520,1,GeoData Consultants Ltd,47520
public_meeting,44831,2,True,40743
permit,45081,2,True,31028
source_class,47520,3,groundwater,36627
management_group,47520,5,user-group,42018
quantity_group,47520,5,enough,26538
quantity,47520,5,enough,26538
waterpoint_type_group,47520,6,communal standpipe,27615
quality_group,47520,6,good,40633
payment_type,47520,7,never pay,20318


In [8]:
X_train.management.value_counts()

vwc                 32455
wug                  5204
water board          2326
wua                  2033
private operator     1566
parastatal           1413
water authority       716
other                 683
company               524
unknown               456
other - school         81
trust                  63
Name: management, dtype: int64

In [9]:
X_train.scheme_management.value_counts()
# strange that the unique values are so similiar between these two features, 
# yet their value counts are markedly different.

VWC                 29462
WUG                  4161
Water authority      2522
WUA                  2312
Water Board          2175
Parastatal           1346
Private operator      862
Company               820
Other                 626
SWC                    75
Trust                  56
None                    1
Name: scheme_management, dtype: int64

In [10]:
X_train.management_group.value_counts()

user-group    42018
commercial     2869
parastatal     1413
other           764
unknown         456
Name: management_group, dtype: int64

In [11]:
X_train[(X_train['management'] == 'wug') & (X_train['scheme_management'] != 'WUG')].head()

# It's not clear why the management of the pumps seems inconsistently coded.
# But management_group and management seems to agree (see count for parastatal
# in each) so I'm going to use 'management'.

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
13883,37803,0.0,2012-10-07,Rwssp,0,DWE,32.255165,-4.135612,Shuleni,0,Lake Tanganyika,Ibambara A,Shinyanga,17,3,Kahama,Ushetu,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14343,8353,300.0,2011-03-21,Co,300,Co,36.997532,-7.599857,Kwa Mgoe,0,Rufiji,Kisiwani,Morogoro,5,1,Kilosa,Kidodi,60,True,GeoData Consultants Ltd,VWC,It,True,2011,gravity,gravity,gravity,wug,user-group,pay when scheme fails,on failure,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
12789,22298,0.0,2013-02-02,World Vision,0,TAWASA,32.516037,-3.762045,Matalange,0,Lake Tanganyika,Matalange,Shinyanga,17,3,Kahama,Ngongwa,0,False,GeoData Consultants Ltd,NaN,NaN,True,0,other,other,other,wug,user-group,unknown,unknown,milky,milky,seasonal,seasonal,shallow well,shallow well,groundwater,other,other
27714,54718,0.0,2011-03-27,Village Council,0,Village Council,33.313971,-9.009151,Kajobile,0,Lake Rukwa,Nsambya,Mbeya,12,2,Mbeya Rural,Iwindi,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,gravity,gravity,gravity,wug,user-group,pay when scheme fails,on failure,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
28420,61737,500.0,2011-03-21,Caritas,356,DWE,36.429111,-8.657954,Kwa Bomola,0,Rufiji,Mahimbo Juu,Morogoro,5,4,Ulanga,Itete,290,True,GeoData Consultants Ltd,Water Board,Itete wa,True,2006,gravity,gravity,gravity,wug,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [0]:
import seaborn as sns

In [27]:
# Subset non-numerical
X_train_categorical = X_train.select_dtypes(exclude='number')
X_val_categorical = X_val.select_dtypes(exclude='number')

categorical_features = []
for i in X_train_categorical.columns:
  if X_train[i].nunique() < 22:
    if X_train[i].isnull().sum() == 0:
      categorical_features.append(i)
    else:
      print(i, X_train[i].isnull().sum())

# Not using features with nulls in our regression.
print(categorical_features)

public_meeting 2689
scheme_management 3102
permit 2439
['basin', 'region', 'recorded_by', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [0]:
# categorical_features.append('scheme_management')
# #Let's see if this improves our model, even leaving in the NaNs

In [0]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from category_encoders import OneHotEncoder

numerical_features = X_train_numeric.columns.drop('id').tolist()

In [49]:
print(numerical_features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']


In [0]:
features = categorical_features + numerical_features

X_train_subset = X_train[features]
X_val_subset = X_val[features]

encoder = OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.fit_transform(X_val_encoded)

model1 = LogisticRegression(n_jobs=-1)
model2 = LogisticRegressionCV(n_jobs=-1)

model1.fit(X_train_scaled, y_train)
model2.fit(X_train_scaled, y_train)

print("LR Validation accuracy", model1.score(X_val_scaled, y_val))
print("LRCV Validation accuracy", model2.score(X_val_scaled, y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarni

LR Validation accuracy 0.7312289562289562
LRCV Validation accuracy 0.731986531986532


In [52]:
model1.coef_, model2.coef_

# I don't know what plotting these coefficients would entail.

(array([[ 1.96309019e-02, -9.18780363e-02,  1.87742451e-02,
          2.30369751e-02, -4.33337395e-02,  1.38349554e-02,
         -9.95166715e-02,  2.03167620e-01, -3.94656188e-02,
          9.81311422e-02, -2.76050153e-02, -2.89124516e-01,
         -8.14680499e-02,  2.04471781e-01,  1.01199071e-01,
         -6.17831237e-02,  1.01498015e-01,  5.82307143e-02,
          2.16151330e-01,  8.16500344e-02,  7.34503437e-02,
         -1.01193892e-02, -1.07876272e-01, -7.56655900e-02,
         -1.21746166e-01, -3.74372489e-02,  8.25240148e-02,
         -1.45481226e-01, -1.65661380e-01,  1.18616976e-02,
          0.00000000e+00, -4.78479310e-02,  5.43774031e-02,
         -7.78464034e-02,  1.27711953e-01,  5.85648193e-03,
         -3.05812342e-02,  3.09339127e-02,  4.62770174e-02,
         -1.15896785e-01, -5.85050367e-03,  1.68495381e-02,
          3.40282039e-03, -1.16671602e-02, -2.50012611e-02,
         -7.80982250e-03, -4.93507059e-03, -2.08738964e-03,
         -4.26809749e-02, -4.78479310e-0